## 2012-2017 사이 개봉영화 페이지 파일 저장

In [4]:
import requests

base = 'http://www.kobis.or.kr/kobis/business/stat/offc/searchOfficHitTotList.do?'

for i in range(1,687):
    param = 'curPage=%s&dmlMode=search&searchMode=year&searchType=&point=&standYySt=2012&standYyEd=2017&repNation=&movieNm=&openDtFr=2012-01-01&openDtTo=2017-11-30&multiMovieYn=&planType=018503&orderCond=asc&orderBy=1' % i
    req = requests.get(base,params=param)
    
    f = open('./movie_lists_html/%s.html'% i, 'w')
    f.write(req.content)
    f.close()
    print(i,end='\r')

ConnectionError: HTTPConnectionPool(host='www.kobis.or.kr', port=80): Max retries exceeded with url: /kobis/business/stat/offc/searchOfficHitTotList.do?curPage=1&dmlMode=search&searchMode=year&searchType=&point=&standYySt=2012&standYyEd=2017&repNation=&movieNm=&openDtFr=2012-01-01&openDtTo=2017-11-30&multiMovieYn=&planType=018503&orderCond=asc&orderBy=1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7ff99e982d68>: Failed to establish a new connection: [Errno 110] Connection timed out',))

## 각 파일 열어서 이쁜수프

In [1]:
%%time
import os
from bs4 import BeautifulSoup
import lxml
import re
import pandas as pd

filelist = os.listdir('./movie_lists_html/')
filelist.sort()

movie_df = pd.DataFrame(columns=['movie_name','movie_code','director_name','director_code','release_date','nation'])
movie_df_len = 0

for files in filelist:
    
    file = open('./movie_lists_html/%s' % files)
    
    soup = BeautifulSoup(file,'lxml')

    table = soup.find('table',border='1')
    tbody = soup.find('tbody')
    
    for tr in tbody.find_all('tr'): ## html 테이블 한 행 씩
        director_code = []
        director_name = []
        temp = []
        for a in tr.find_all('a'):
            #### 영화코드
            if 'movie' in a['onclick']:
                movie_code = (re.search(r'\d+',a['onclick']).group()) ### 영화 코드만 걸르기
                movie_name = a.string

            #### 감독코드 !! 감독은 여럿일 수 있음
            if 'people' in a['onclick']:
                director_code.append(re.search(r'\d+', a['onclick']).group())
                director_name.append(a.string)
        ### 개봉일
        for td in tr.find_all('td',limit=6):
            release_year = td.string

        ### 국적
        for td in tr.find_all('td',limit=9):
            nation = td.string
            
        ##감독 이름,코드 , 로 조인
        dir_name = '|'.join(director_name)
        dir_code = '|'.join(director_code)
        
        movie_df.loc[movie_df_len] = [movie_name,
                                      movie_code,
                                      dir_name,
                                      dir_code,
                                      release_year,
                                      nation
                                     ]
        movie_df_len+=1
    file.close()

In [2]:
movie_df.to_csv('./2012-2017 개봉영화.csv', encoding='utf-8')

## 1, 8일차 누적관객 BS

### 전체 받기
http://kobis.or.kr/kobis/business/mast/mvie/searchMovieDtlXls.do?code=20140110&sType=box

In [1]:
%%time
import os
from bs4 import BeautifulSoup
import lxml
import re
import pandas as pd
import datetime

filelist = os.listdir('./movie_box/')

movie_12_17 = pd.read_csv('./2012-2017 개봉영화.csv', encoding='utf-8')

def day8_convert(release_date):
    con_release_date = datetime.datetime.strptime(release_date, "%Y-%m-%d").date()
    day8_release_date = con_release_date + datetime.timedelta(days=7)
    day8 = day8_release_date.strftime('%Y-%m-%d')
    return day8


def day15_convert(release_date):
    con_release_date = datetime.datetime.strptime(release_date, "%Y-%m-%d").date()
    day15_release_date = con_release_date + datetime.timedelta(days=13)
    day15 = day15_release_date.strftime('%Y-%m-%d')
    return day15

movie_day_df = pd.DataFrame(columns=['movie_name',
                                     'movie_code',
                                     'director_name',
                                     'director_code',
                                     'nation',
                                     'release_date',
                                     'day1_date',
                                     'day1_screen',
                                     'day1_audience',
                                     'day8_date',
                                     'day8_audience',
                                     'day15_date',
                                     'day15_audience'
                                    ])
df_row = 0
for row in movie_12_17.itertuples():
    try:
        movie_name = getattr(row,'movie_name')
        movie_code = getattr(row,'movie_code')
        director_name = getattr(row,'director_name')
        director_code = getattr(row,'director_code')
        nation = getattr(row,'nation')
        release_date = getattr(row,'release_date') ## %Y-%m-%d

        file = open('./movie_box/%s.html' % movie_code)
        soup = BeautifulSoup(file,'lxml')

        table = soup.find('table', class_='boardList02')
        tbody = table.find('tbody')

        day8_assume = day8_convert(release_date) ## 8일차 날짜
        day15_assume = day15_convert(release_date) ## 15일차 날짜

        day1_screen = 0
        day1_audience = 0
        day8_audience = 0
        day8_screen = 0
        day15_screen = 0
        day15_audience = 0


        for tr in tbody.find_all('tr'): ## 여기서 한줄씩
            tr_list = []
            for td in tr.find_all('td'): ## 한 데이터씩
                tr_list.append(td.getText())
            ################################################### 개봉일일때
            if(release_date == tr_list[0]):
                day1_date = release_date ### 개봉일
                day1_screen = tr_list[1] ### 스크린수
                day1_audience = (tr_list[9]).replace(',','') ### 당일 관객

            elif(day8_assume == tr_list[0]):
                day8_screen = tr_list[1]
                day8_audience = (tr_list[12]).replace(',','')

            elif(day15_assume == tr_list[0]):
                day15_screen = tr_list[1]
                day15_audience = (tr_list[12]).replace(',','')
        movie_day_df.loc[df_row]=[movie_name,
                                  movie_code,
                                  director_name,
                                  director_code,
                                  nation,
                                  release_date,
                                  day1_date,
                                  day1_screen,
                                  day1_audience,
                                  day8_assume,
                                  day8_audience,
                                  day15_assume,
                                  day15_audience
                                 ]
        df_row +=1
        print(df_row,end='\r')
    except:
        print(movie_code)
        pass

2010
CPU times: user 4min 33s, sys: 2.17 s, total: 4min 35s
Wall time: 4min 37s


In [4]:
movie_day_df.to_csv('./day8_14.csv', encoding='utf-8')